In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from pathlib import Path

from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from torch_uncertainty.routines import ClassificationRoutine
from torch_uncertainty.utils import TUTrainer
from lightning.pytorch import Trainer
from torch_uncertainty.models import deep_ensembles, mc_dropout
from torch_uncertainty.transforms import RepeatTarget
import torchvision.transforms as T

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm


In [11]:
results_dict = {}

labeling_functions=[
    "guo_los",
    "guo_readmission",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_pancan",
    "new_celiac",
    "new_lupus",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia" # will OOM at 200G on `gpu` partition
]

class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p=0.4)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.dropout2 = nn.Dropout(p=0.4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return x

def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05 * lr_mult)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    return {"optimizer": optimizer, "scheduler": scheduler}

max_epochs = 50
batch_size = 64

for i in tqdm(range(len(labeling_functions))):
# for i in [0]:

    task = labeling_functions[i]
    folder_path = f'single_task_data/{task}'

    train_x_name = os.path.join(folder_path, 'X_train.csv')
    train_y_name = os.path.join(folder_path, 'y_train.csv')
    val_x_name = os.path.join(folder_path, 'X_val.csv')
    val_y_name = os.path.join(folder_path, 'y_val.csv')
    test_x_name = os.path.join(folder_path, 'X_test.csv')
    test_y_name = os.path.join(folder_path, 'y_test.csv')

    X_train = pd.read_csv(train_x_name).to_numpy()
    y_train = pd.read_csv(train_y_name).to_numpy().reshape(-1)
    X_val = pd.read_csv(val_x_name).to_numpy()
    y_val = pd.read_csv(val_y_name).to_numpy().reshape(-1)
    X_test = pd.read_csv(test_x_name).to_numpy()
    y_test = pd.read_csv(test_y_name).to_numpy().reshape(-1)

    # Create class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    X_train = torch.tensor(X_train).float()
    X_val = torch.tensor(X_val).float()
    X_test = torch.tensor(X_test).float()
    y_train = torch.tensor(y_train).long()
    y_val = torch.tensor(y_val).long()
    y_test = torch.tensor(y_test).long()

    # Create TensorDatasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoaders
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    input_size = X_train.shape[1]
    hidden_size = 128
    num_classes = 2

    model = TwoLayerNet(768, 128, 2)

    mc_model = mc_dropout(model, num_estimators=5, last_layer=False, on_batch=False)

    # The routine is a wrapper of the model that contains the training logic with the metrics, etc
    routine = ClassificationRoutine(
        num_classes=2,
        model=mc_model,
        loss=nn.CrossEntropyLoss(),
        optim_recipe=optim_recipe(mc_model),
        is_ensemble=True,
    )
    trainer = Trainer(accelerator="gpu", max_epochs=max_epochs, enable_progress_bar=False)

    trainer.fit(model=routine, train_dataloaders=train_dl, val_dataloaders=val_dl)

    # routine.model.on_batch = True
    results = trainer.test(model=routine, dataloaders=[test_dl])

    results_dict[task] = results

  0%|          | 0/14 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 98.7 K | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | Entropy          | 0      | train
6 | test_id_ens_metrics | MetricCollection | 0      | train
7 | mixup               | Identity         | 0      | train
-----------------------------------

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │   0.056588202714920044    │
│       test/cal/aECE       │    0.08273766934871674    │
│       test/cls/Acc        │    0.7653758525848389     │
│      test/cls/Brier       │    0.34179937839508057    │
│       test/cls/NLL        │    0.5204537510871887     │
│     test/cls/entropy      │    0.5351939797401428     │
│   test/ens_Disagreement   │    0.19435077905654907    │
│     test/ens_Entropy      │    0.39640647172927856    │
│        test/ens_MI        │    0.13878746330738068    │
│       test/sc/AUGRC       │    0.08739209920167923    │
│       test/sc/AURC        │    0.14651180803775787    │
│    test/sc/CovAt5Risk     │            nan            │
│    test/sc/RiskAt80Cov    │    0.19988609850406647    │
└───────────────────────────┴───────────────────────────┘

  7%|▋         | 1/14 [00:10<02:11, 10.10s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 98.7 K | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | Entropy          | 0      | train
6 | test_id_ens_metrics | MetricCollection | 0      | train
7 | mixup               | Identity         | 0      | train
---------------------------

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.1194390133023262     │
│       test/cal/aECE       │    0.1124759390950203     │
│       test/cls/Acc        │    0.8752855062484741     │
│      test/cls/Brier       │    0.24314047396183014    │
│       test/cls/NLL        │    0.41424497961997986    │
│     test/cls/entropy      │    0.4885139465332031     │
│   test/ens_Disagreement   │    0.08113294839859009    │
│     test/ens_Entropy      │    0.3353935778141022     │
│        test/ens_MI        │    0.15312044322490692    │
│       test/sc/AUGRC       │    0.04745379835367203    │
│       test/sc/AURC        │    0.0844392254948616     │
│    test/sc/CovAt5Risk     │            nan            │
│    test/sc/RiskAt80Cov    │    0.09417808055877686    │
└───────────────────────────┴───────────────────────────┘

 14%|█▍        | 2/14 [00:20<02:02, 10.19s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 98.7 K | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | Entropy          | 0      | train
6 | test_id_ens_metrics | MetricCollection | 0      | train
7 | mixup               | Identity         | 0      | train
---------------------------

...

 86%|████████▌ | 12/14 [16:08<06:47, 203.76s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 98.7 K | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | Entropy          | 0      | train
6 | test_id_ens_metrics | MetricCollection | 0      | train
7 | mixup               | Identity         | 0      | train
-------------------------

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.13507552444934845    │
│       test/cal/aECE       │    0.1321554183959961     │
│       test/cls/Acc        │    0.8853752613067627     │
│      test/cls/Brier       │    0.21865417063236237    │
│       test/cls/NLL        │    0.3756517171859741     │
│     test/cls/entropy      │    0.5064991116523743     │
│   test/ens_Disagreement   │    0.3533281981945038     │
│     test/ens_Entropy      │    0.3585791289806366     │
│        test/ens_MI        │    0.14791974425315857    │
│       test/sc/AUGRC       │    0.03078387677669525    │
│       test/sc/AURC        │    0.04892715439200401    │
│    test/sc/CovAt5Risk     │    0.5513370037078857     │
│    test/sc/RiskAt80Cov    │    0.06764250993728638    │
└───────────────────────────┴───────────────────────────┘

 93%|█████████▎| 13/14 [20:41<03:44, 224.98s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 98.7 K | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         | 0      | train
3 | val_cls_metrics     | MetricCollection | 0      | train
4 | test_cls_metrics    | MetricCollection | 0      | train
5 | test_id_entropy     | Entropy          | 0      | train
6 | test_id_ens_metrics | MetricCollection | 0      | train
7 | mixup               | Identity         | 0      | train
-------------------------

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.1907128095626831     │
│       test/cal/aECE       │    0.1906720995903015     │
│       test/cls/Acc        │    0.9845079779624939     │
│      test/cls/Brier       │    0.12376556545495987    │
│       test/cls/NLL        │    0.2643277645111084     │
│     test/cls/entropy      │    0.46321433782577515    │
│   test/ens_Disagreement   │   0.007875273004174232    │
│     test/ens_Entropy      │    0.28954923152923584    │
│        test/ens_MI        │    0.17366530001163483    │
│       test/sc/AUGRC       │   0.006073290947824717    │
│       test/sc/AURC        │   0.011787323281168938    │
│    test/sc/CovAt5Risk     │            1.0            │
│    test/sc/RiskAt80Cov    │   0.012106146663427353    │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 14/14 [28:42<00:00, 123.02s/it]


In [4]:
mc_model.training

False

In [ ]:
x, y = next(iter(train_dl))

In [ ]:
y1 = mc_model(x)

In [ ]:
mc_model.training

True

In [ ]:
y2 = mc_model.core_model(x)

In [ ]:
y2.shape

torch.Size([64, 2])

In [10]:
results

[{'test/cal/ECE': 0.15296123921871185,
  'test/cal/aECE': 0.1529611051082611,
  'test/cls/Acc': 0.7357630729675293,
  'test/cls/Brier': 0.4027252197265625,
  'test/cls/NLL': 0.58946692943573,
  'test/sc/AUGRC': 0.09242186695337296,
  'test/sc/AURC': 0.1380428969860077,
  'test/sc/CovAt5Risk': nan,
  'test/sc/RiskAt80Cov': 0.23120729625225067,
  'test/cls/entropy': 0.659686267375946,
  'test/ens_Disagreement': 0.13321183621883392,
  'test/ens_Entropy': 0.6369427442550659,
  'test/ens_MI': 0.02274354360997677}]

In [4]:
from torch_uncertainty.datamodules import MNISTDataModule
root = Path("data")
datamodule = MNISTDataModule(root=root, batch_size=128)

In [5]:
type(mc_model)

torch_uncertainty.models.wrappers.mc_dropout.MCDropout

In [6]:
datamodule

In [12]:
import json
with open('results_single_mc_dropout.json', 'w') as f:
    json.dump(results_dict, f)